In [1]:
import polars as pl
from src.rag_dme import extract_datasets, GenerativeServiceClient, ChromaPersist
from pathlib import Path
from time import perf_counter
from tqdm import tqdm

/mnt/d/datta/Documents/Github/jcdl2024/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


in module


In [2]:
chromaDB = ChromaPersist(name="embeds", path=Path(".local"))

In [3]:
gsc = GenerativeServiceClient()

I0000 00:00:1722333071.114164     889 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [4]:
dmdd = pl.read_json(".local/queries.json")[350:]

In [5]:
result = []

In [6]:
for paper_id, content in tqdm(dmdd[["paper_id", "content"]][1:2].transpose().to_dict().values()):
    try:
        start = perf_counter()
        datasets = extract_datasets(
                    chromaDB=chromaDB, 
                    gsc=gsc, 
                    full_text=content
                )
        end = perf_counter()
        result.append(
            {
                "paper_id": paper_id, 
                "observed": datasets,
                "time_elapsed": end-start
            }
        )
    except:
        continue

100%|██████████| 1/1 [00:47<00:00, 47.10s/it]


In [7]:
result

[{'paper_id': '209202273',
  'observed': ['LSUN CAR',
   'LSUN HORSE',
   'LSUN CAT',
   'FFHQ',
   'LSUN CHURCH',
   'LSUN'],
  'time_elapsed': 49.18447363200002}]

In [13]:
pl.read_ndjson(".local/350-450.jsonl")[1]["observed"][0]

'FFHQ, Cars, LPIPS, CAT, CHURCH, StyleGAN, CAR, LSUN CAR, LSUN, HORSE'

In [10]:
from pprint import pprint


pprint(dmdd["content"][1])

('The style-based GAN architecture (StyleGAN) yields state-of-the-art results '
 'in data-driven unconditional generative image modeling. We expose and '
 'analyze several of its characteristic artifacts, and propose changes in both '
 'model architecture and training methods to address them. In particular, we '
 'redesign the generator normalization, revisit progressive growing, and '
 'regularize the generator to encourage good conditioning in the mapping from '
 'latent codes to images. In addition to improving image quality, this path '
 'length regularizer yields the additional benefit that the generator becomes '
 'significantly easier to invert. This makes it possible to reliably attribute '
 'a generated image to a particular network. We furthermore visualize how well '
 'the generator utilizes its output resolution, and identify a capacity '
 'problem, motivating us to train larger models for additional quality '
 'improvements. Overall, our improved model redefines the state 